<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_tagging/life_stages/insect_lifestages_yolov4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using YOLO v3 pre-trained on Google Open Images to detect insect life stages (juvenile, adult) from EOL images
---
*Last Updated 22 February 2021*   
Using a YOLOv3 model pre-trained on Google Open Images as a method to do customized, large-scale image processing. Insect images will be tagged using the location and dimensions of the detected life stages to improve search features of EOL.

# Installs
---

In [ ]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# clone darknet repo
%cd /content/drive/My Drive/train/darknet
#!git clone https://github.com/AlexeyAB/darknet

# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# verify CUDA
!/usr/local/cuda/bin/nvcc --version

# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

# download pre-trained weights (only run once)
#!wget https://pjreddie.com/media/files/yolov3-openimages.weights

# For importing/exporting files, working with arrays, etc
import os
import pathlib
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import numpy as np 
import csv
import matplotlib.pyplot as plt
import time
import pandas as pd

# For downloading the images
!apt-get install aria2

# For drawing onto and plotting the images
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import cv2

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Classify images
---

### Temporarily download images from EOL bundle to Google Drive (YOLO cannot directly parse URL images)

In [ ]:
# Download images
bundle = "https://editors.eol.org/other_files/bundle_images/files/images_for_Lepidoptera_20K_breakdown_download_000001.txt" #@param {type:"string"}
df = pd.read_csv(bundle)

# Take subset of bundle
# TO DO: Change file name for each bundle/run abcd if doing 4 batches using dropdown form to right
ss = "life_stage_tags_c" #@param ["life_stage_tags_a", "life_stage_tags_b", "life_stage_tags_c", "life_stage_tags_d"] {allow-input: true}
ss = ss + ".txt"

# Run in 4 batches of 5k images each (batch a is from 0-5000, b from 5000 to 10000, etc)
if "a" in ss:
  a=0
  b=5000
elif "b" in ss:
  a=5000
  b=10000
elif "c" in ss:
  a=10000
  b=15000
elif "d" in ss:
  a=15000
  b=20000

# Save subset to text file for image download
df = df.iloc[a:b]
outpath = "/content/drive/My Drive/train/darknet/data/imgs/" + ss
df.to_csv(outpath, sep='\n', index=False, header=False)

# Download images (takes 7-10 min per 5k imgs, aria2 downloads 16imgs at a time)
%cd /content/drive/My Drive/train/darknet/data/imgs
!aria2c -x 16 -s 1 -i $ss

# Check how many images downloaded
print("Number of images downloaded to Google Drive: ")
!ls . | wc -l

In [ ]:
# Move text file to image_data/bundles
%cd ../
!mv imgs/*.txt img_info/

In [ ]:
# Make imgs.txt file to run images through YOLO for inference in batches
%cd /content/drive/My Drive/train/darknet/data/

import glob
import os

path = "/content/drive/My Drive/train/darknet/data/imgs"
inf_ss = path+'/'+ss
with open(inf_ss, 'w', encoding='utf-8') as f:
  for dir, dirs, files in os.walk(path):
    files = [fn for fn in files]
    for fn in files:
      if 'txt' not in fn:
        out = "data/imgs/" + fn
        f.writelines(out + '\n')

# Inspect imgs.txt file to confirm length and content
df = pd.read_csv(inf_ss, header=None)
print(df.head())
print(len(df))

### Run images through trained model

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# TO DO: In next bloc, change inference image file list name at end of line after "<" to match inf_ss defined above
# ex: data/imgs/plant_poll_coocc_tags_a.txt
print("filename to copy-paste into code block below:", os.path.basename(inf_ss))

In [ ]:
# TO DO: Change inference image file list name at end of line after "<" to match inf_ss defined above
%cd /content/drive/My Drive/train/darknet

# darknet run with external output flag to print bounding box coordinates
!./darknet detector test cfg/openimages.data cfg/yolov3-openimages.cfg yolov3-openimages.weights -dont_show -save_labels < data/imgs/life_stage_tags_c.txt

# Post-process model output
---

In [ ]:
# Combine individual prediction files for each image to all_predictions.txt

# Delete image file list for inference
path = "/content/drive/My Drive/train/darknet/data/imgs"
inf_ss = path+'/'+ss
inf_ss = "data/imgs/"+ss
!rm $inf_ss

# Combine individual text files and image filenames into all_predictions.txt
fns = os.listdir('data/imgs')
with open('data/results/all_predictions.txt', 'w') as outfile:
  header = "class_id x y w h img_id"
  outfile.write(header + "\n")
  for fn in fns:
        if 'txt' in fn:
          with open('data/imgs/'+fn) as infile:
            lines = infile.readlines()
            newlines = [''.join([x.strip(), ' ' + os.path.splitext(fn)[0] + '\n']) for x in lines]
            outfile.writelines(newlines)

# Inspect saved predictions
df = pd.read_csv('data/results/all_predictions.txt')
print(df.head())

# Delete all individual prediction files
!rm -r data/imgs/*.txt

# Delete all image files now that they have been used for inference
!rm -r data/imgs/*

In [ ]:
# Create final predictions dataframe with class names (instead of numbers) and image urls
# EOL image url bundle
df = pd.read_csv(bundle)
df.columns = ['url']
print(df)

# Model predictions with number-coded classes
predict = pd.read_csv('data/results/all_predictions.txt', header=0, sep=" ")
predict.class_id = predict.class_id - 1 #class_id counts started from 1 instead of 0 from YOLO
print(predict)

# Add class names to model predictions
classnames = pd.read_table('data/openimages.names')
classnames.columns = ['classname']
#print(classnames)
tag_df = predict.copy()
di = pd.Series(classnames.classname.values,index=classnames.index).to_dict()
tag_df.replace({"class_id":di}, inplace=True)
tag_df['class_id'] = tag_df['class_id'].astype(str)
print(tag_df)

# Filter for desired classes
filter = ['Butterfly', 'Insect', 'Beetle', 'Ant', 'Bat (Animal)', 'Bird', 'Bee', \
          'Invertebrate']

# Add urls to model predictions
map_urls = df.copy()
img_ids = map_urls['url'].apply(lambda x: os.path.splitext((os.path.basename(x)))[0])
map_urls['img_id'] = img_ids
#print(map_urls)

tag_df.set_index('img_id', inplace=True, drop=True)
map_urls.set_index('img_id', inplace=True, drop=True)
mapped_tagdf = tag_df.merge(map_urls, left_index=True, right_index=True)
mapped_tagdf.reset_index(drop=False, inplace=True)
mapped_tagdf.drop_duplicates(inplace=True, ignore_index=True)
print(mapped_tagdf.head())

# Save final tags to file
fn = os.path.splitext(os.path.basename(inf_ss))[0]
outpath = 'data/results/' + fn + '.tsv'
mapped_tagdf.to_csv(outpath, sep="\t", index=False)

# Display predictions on images
---
Inspect detection results and verify that they are as expected

In [ ]:
# TO DO: Do you want to use the tagging file exported above?
use_outpath = "yes" #@param ["yes", "no"]
# If no, choose other path to use
otherpath = "\u003Cpath to other tag file> " #@param {type:"string"}
if use_outpath == "yes":
  outpath = outpath
else:
  outpath = otherpath
df = pd.read_csv(outpath, sep="\t", header=0)

# For uploading an image from url
# Modified from https://www.pyimagesearch.com/2015/03/02/convert-url-to-image-with-python-and-opencv/
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
  return image

In [ ]:
# Display crop dimensions on images
# Adjust line below to see up to 50 images displayed at a time
for i, row in df.iloc[0:5].iterrows():
  # Read in image 
  url = df['url'][i]
  img = url_to_image(url)
  h,w = img.shape[:2]
  # Define variables needed to draw bounding box on image
  xmin = round((df['x'][i] - (df['w'][i]/2))*w)
  if (xmin < 0): xmin = 0
  ymin = round((df['y'][i] - (df['h'][i]/2))*h)
  if (ymin < 0): ymin = 0
  xmax = round(xmin + (df['w'][i]) * w)
  if (xmax > w-1): xmax = w-1
  ymax = round(ymin + (df['h'][i].astype(int)) * h)
  if (ymax > 0): ymax = h-1

  # Set box/font color and size
  maxdim = max(df['w'][i],df['h'][i])
  fontScale = 1

  box_col = (255, 0, 157)
  
  # If using multitaxa dataset, draw color-coded boxes and class labels on images
  if 'class_id' in df:
    taxon = df['class_id'][i]
    if taxon == "Ladybug":
      box_col = (255,199,15)
    elif taxon == "Beetle":
      box_col = (255,127,0)
    elif taxon == "Insect":
      box_col = (255,42,22)

    # Draw taxon label on image
    image_wbox = cv2.putText(img, taxon, (xmin+7, ymax-12), cv2.FONT_HERSHEY_SIMPLEX, fontScale, box_col, 2, cv2.LINE_AA)
  
  # Draw box on image
  image_wbox = cv2.rectangle(img, (xmin, ymax), (xmax, ymin), box_col, 5)
  
  # Plot and show cropping boxes on images
  _, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(image_wbox)
  # Display image URL above image to facilitate troubleshooting/fine-tuning of data reformatting and tidying steps in convert_bboxdims.py or preprocessing.ipynb
  plt.title('{}'.format(url, xmin, ymin, xmax, ymax))